In [1]:
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
import requests
import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from mpire import WorkerPool



In [2]:
# # Documentation

# Retrieveing data from spotify
# https://medium.com/@rafaelnduarte/how-to-retrieve-data-from-spotify-110c859ab304
# https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
    
# Spotipy documentation
# https://spotipy.readthedocs.io/en/2.9.0/#
    
# Spotipy examples
# https://github.com/plamere/spotipy/tree/master/examples

# Great analysis of audio data
# https://medium.com/@FinchMF/praise-questions-and-critique-spotify-api-38e984a4174b

# # Other API IDeas
# Breaking news and yahoo finance
# Deezer
# Spotify
# https://rapidapi.com/search/spotify
# https://rapidapi.com/theaudiodb/api/theaudiodb
# https://rapidapi.com/dimashirokov/api/LastFm


In [3]:
client_id = '624ce7dd9a424b858e2ea4e3b19229fc'
client_secret = 'd6713dd18f0345e9b1709ab8420daa66'

# In the code above, replace the Client ID and Client Secret variables with your own and make sure they are inside quotes.

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
response = sp.search(q='year:2020', type='track', limit=50)
len(response.get('tracks').get('items'))

50

In [5]:
response.get('tracks').get('items')[1].get('id')

'0MlkZnfwsjwHHSfn07N8ao'

In [6]:
# create empty lists where the results are going to be stored
def retrieve_track_ids(year):
    list_ = []
    for i in range(0,100000,50):
        try:
            track_results = sp.search(q=f'year:{year}', type='track', limit=50,offset=i, market='US')
            for i in range(len(track_results.get('tracks').get('items'))):
                list_.append(track_results.get('tracks').get('items')[i].get('id'))
            time.sleep(.5)
        except:
            break
    return list_

In [9]:
%%time

with WorkerPool(n_jobs=5) as pool:
    year_list = list(np.arange(2000,2022))

    track_ids = []

    for year in year_list:
        holder_list = []
        holder_list = retrieve_track_ids(year)
        track_ids.extend(holder_list)

HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.c

Wall time: 5min 9s


In [10]:
len(track_ids)

22000

In [8]:
len(track_ids)

22000

In [11]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)
    audio_analysis = sp.audio_analysis(id)
    
    artist_url = meta['artists'][0]["external_urls"]["spotify"]
    artist = sp.artist(artist_url)

    album_url = meta['album']["external_urls"]["spotify"]
    album = sp.album(album_url)
    
    # album and artist info
    artist_genres = artist["genres"]
    artist_popularity = artist["popularity"]
    artist_no_of_followers = artist["followers"].get('total')
    artist_type = artist["type"]

    album_label = album['label']
    album_popularity = album['popularity']
  
    # meta
    song_id = id 
    song_name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    song_release_date = meta['album']['release_date']
    length = meta['duration_ms']
    song_popularity = meta['popularity']

    # features
    key = features[0]['key']
    mode = features[0]['mode']
    acousticness = features[0]['acousticness']
    valence = features[0]['valence']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    
    tempo_confidence = audio_analysis.get('track').get('tempo_confidence')
    key_confidence = audio_analysis.get('track').get('key_confidence')
    time_signature_confidence = audio_analysis.get('track').get('time_signature_confidence')
    mode_confidence = audio_analysis.get('track').get('mode_confidence')
    rhythm_version = audio_analysis.get('track').get('rhythm_version')
    synch_version = audio_analysis.get('track').get('synch_version')
    number_of_segments = len(audio_analysis.get('segments'))
    number_of_bars = len(audio_analysis.get('bars'))
    number_of_beats = len(audio_analysis.get('beats'))
    number_of_sections = len(audio_analysis.get('sections'))
    number_of_tatums = len(audio_analysis.get('tatums'))

    
    track = [song_id, song_name, album, artist, artist_genres, artist_popularity, artist_no_of_followers, artist_type
             ,album_label, album_popularity, song_release_date, length, song_popularity, key, mode
             ,acousticness,valence, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo
             ,time_signature, tempo_confidence, key_confidence, time_signature_confidence, mode_confidence
             ,rhythm_version, synch_version, number_of_segments, number_of_bars, number_of_beats, number_of_sections
            ,number_of_tatums 
]
    return track

In [12]:
%%time

# loop over track ids 
tracks = []

with WorkerPool(n_jobs=5) as pool:
    for i in range(len(track_ids)):
    #     time.sleep(.5)
        try:
            track = getTrackFeatures(track_ids[i])
            tracks.append(track)
        except:
            continue

        
# create dataset
df = pd.DataFrame(tracks, columns = [
    'song_id', 'song name', 'album', 'artist', 'artist genres', 'artist popularity', 'artist number of followers'
    ,'artist type', 'album label', 'album popularity', 'song release date', 'length', 'song popularity'
    ,'key', 'mode', 'acousticness','valence', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness'
    ,'speechiness','tempo', 'time signature', 'tempo confidence', 'key confidence', 'time signature confidence'
    ,'mode confidence','rhythm version', 'synch version', 'number of segments', 'number of bars', 'number of beats'
    ,'number of sections', 'number of tatums' 
])

# Add a date of datapull
today_timestamp = pd.to_datetime("today")
today_date = today_timestamp.date()
df['date of data pull'] = today_date
df['date of data pull'] = pd.to_datetime(df['date of data pull'])

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

artist_genres_dummies = pd.DataFrame(mlb.fit_transform(df['artist genres']),columns=mlb.classes_, index=df.index)

final_df = pd.concat([df,artist_genres_dummies], axis = 1)

final_df.to_csv(f"spotify_{today_date}.csv")

HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/0rWrhQIdbzIb24obQil8Wi returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/6MFQeWtk7kxWGydnJB2y36 returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/5OGkKx8jP0A5KSULEc6XYZ returned 404 due to analysis not found
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/04boE4u1AupbrGlI62WvoO returned 404 due to analysis not fo

Wall time: 4h 47min


In [13]:
len(tracks)

21936

In [11]:
len(tracks)

21962